### Step 1 Generate the geographical data of Toronto

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium


#import data from a web page and scrape the table
res=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup=BeautifulSoup(res.content,'html.parser')
table=soup.find_all('table')[0]
Mcode_df=pd.read_html(str(table))[0]

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [2]:
#remove 'not assigned' in column 'Borough'
Mcode_df.replace('Not assigned',np.nan,inplace=True)
Mcode_df.dropna(subset=['Borough'],axis=0,inplace=True)
#replace 'not assigned' in column 'Neighbourhood' with the corresponding borough
Mcode_df.fillna(method='ffill',axis=1,inplace=True)
#group neighbourhoods in the same borough and create a clean dataframe can_df
mcode_df=Mcode_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [3]:
#geocoder cannot retrieve the coordinates so the csv file is used
coord_df=pd.read_csv('http://cocl.us/Geospatial_data')
coord_df.rename(columns={'Postal Code':'Postcode'},inplace=True)
#combine two dataframes as a new dataframe
mcode_coord_df=pd.merge(mcode_df,coord_df,on=['Postcode'])
mcode_coord_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### Step 2 Obtain venue information from Foursquare

In [4]:
#Hidden encoding cell
CLIENT_ID = 'K1QQ31GDDLSAXRJ2RCDI134XGVE5G0WNA0H1XW3TZOXV4VGS'
CLIENT_SECRET = 'SO3ADVRZ540GGKK5VI3O4VDTLAXBHDEDLXX0JQXX0KNKTEM5'
VERSION = '20191018'

In [5]:
#define a function to clean the category data
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [6]:
#fetch venue data from Foursquare for each postcode
venues=pd.DataFrame()

for i in range(len(mcode_coord_df)):
    LATITUDE=mcode_coord_df['Latitude'][i]
    LONGITUDE=mcode_coord_df['Longitude'][i]
    url='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,LATITUDE,LONGITUDE,500,100)
    results=requests.get(url).json()
    venues_temp=results['response']['venues']
    venues_temp=json_normalize(venues_temp)
    filtered_columns=['name','location.postalCode','location.lat','location.lng','categories']
    venues_temp=venues_temp.loc[:,filtered_columns]
    venues_temp.dropna(subset=['location.postalCode'],axis=0,inplace=True)
    venues_temp['location.postalCode']=venues_temp['location.postalCode'].astype(str).str[0:3].str.upper()
    venues_temp['categories']=venues_temp.apply(get_category_type,axis=1)
    venues=pd.concat([venues,venues_temp],axis=0,sort=False,ignore_index=True)

venues.rename(columns={'name':'Name','location.postalCode':'Postcode','location.lat':'Venue_Latitude','location.lng':'Venue_Longitude','categories':'Categories'},inplace=True)    
venues_df=pd.merge(venues,mcode_coord_df,on=['Postcode'])
venues_df

,Name,Postcode,Venue_Latitude,Venue_Longitude,Categories,Borough,Neighbourhood,Latitude,Longitude
0,Kaycan - SCARBOROUGH,M1B,43.806768,-79.197875,Building,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,Shell,M1B,43.803227,-79.192414,Gas Station,Scarborough,"Rouge, Malvern",43.806686,-79.194353
2,FASTSIGNS,M1B,43.807882,-79.201968,Business Service,Scarborough,"Rouge, Malvern",43.806686,-79.194353
3,Staples Print & Marketing Services,M1B,43.800175,-79.196921,Print Shop,Scarborough,"Rouge, Malvern",43.806686,-79.194353
4,Hydrografica,M1B,43.804553,-79.199560,Design Studio,Scarborough,"Rouge, Malvern",43.806686,-79.194353
5,Discount Tile,M1B,43.806910,-79.201656,None,Scarborough,"Rouge, Malvern",43.806686,-79.194353
6,Quick Lane Tire & Auto Centre,M1B,43.801101,-79.189315,Auto Garage,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,250 Brenyon Way,M1B,43.797985,-79.198003,Residential Building (Apartment / Condo),Scarborough,"Rouge, Malvern",43.806686,-79.194353
8,Obsidian Mortgage Corporation & Financial Serv...,M1B,43.801100,-79.194131,Bank,Scarborough,"Rouge, Malvern",43.806686,-79.194353
9,Staples Morningside,M1B,43.800285,-79.196607,Paper / Office Supplies Store,Scarborough,"Rouge, Malvern",43.806686,-79.194353


### Step 3 Count restaurants and other venues in each postcode

In [25]:
#one hot encoding
onehot=pd.get_dummies(venues_df[['Categories']], prefix="", prefix_sep="")
onehot['Postcode']=venues_df['Postcode'] 
grouped=onehot.groupby('Postcode').sum().reset_index()

#create subset tables containing japanese restaurants, all restaurants, and other venues
japanese_columns=['Postcode']+['Sushi Restaurant','Japanese Restaurant','Ramen Restaurant']
jp_restaurants_df=grouped[japanese_columns]
jp_restaurants_df.insert(1,'Counts_j',jp_restaurants_df.sum(axis=1))
restaurants_columns=['Postcode']+[col for col in grouped.columns if 'Restaurant' in col or 'Food' in col]
restaurants_df=grouped[restaurants_columns]
restaurants_df.insert(1,'Counts_r',restaurants_df.sum(axis=1))
others_columns=[col for col in grouped.columns if 'Restaurant' not in col and 'Food' not in col]
others_df=grouped[others_columns]
others_df.insert(1,'Counts_o',others_df.sum(axis=1))

#combine subset tables into one and calculate a weighted venue score (-3 for japanese restaurants, -2 for all restaurants, +1 for other venues)
count_df=pd.merge(jp_restaurants_df[['Postcode','Counts_j']],restaurants_df[['Postcode','Counts_r']],on=['Postcode'])
count_df=pd.merge(count_df,others_df[['Postcode','Counts_o']],on=['Postcode'])
count_df.rename(columns={'Counts_j':'No. Japanese restaurants','Counts_r':'No. restaurants','Counts_o':'No. other venues'},inplace=True)
count_df['Score']=count_df['No. Japanese restaurants']*(-3)+count_df['No. restaurants']*(-2)+count_df['No. other venues']*(1)
count_merge_df=pd.merge(mcode_coord_df,count_df,on=['Postcode'])
count_merge_df.sort_values(['Score'],ascending=False,inplace=True)
count_merge_df=count_merge_df.reset_index(drop=True)
count_merge_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude,No. Japanese restaurants,No. restaurants,No. other venues,Score
0,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0,2,69,65
1,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,0,7,67,53
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,1,51,49
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0,2,47,43
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,4,50,42
5,M2N,North York,Willowdale South,43.770120,-79.408493,4,11,75,41
6,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923,1,4,51,40
7,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,0,2,43,39
8,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,1,40,38
9,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,4,46,38


### Step 4 Draw a map for top ranked neighborhoods

In [24]:
map_clusters = folium.Map(location=[43.653963,-79.387207], zoom_start=11)

# add markers to the map
for i in range(5):
    label_top=folium.Popup(str(count_merge_df['Neighbourhood'][i]))
    lat_top=count_merge_df['Latitude'][i]
    lon_top=count_merge_df['Longitude'][i]
    folium.CircleMarker([lat_top,lon_top],radius=5,popup=label_top,color='red',fill=True).add_to(map_clusters)
    label_bottom=folium.Popup(str(count_merge_df['Neighbourhood'][len(count_merge_df)-i-1]))
    lat_bottom=count_merge_df['Latitude'][len(count_merge_df)-i-1]
    lon_bottom=count_merge_df['Longitude'][len(count_merge_df)-i-1]
    folium.CircleMarker([lat_bottom,lon_bottom],radius=5,popup=label_bottom,color='blue',fill=True).add_to(map_clusters)
       
map_clusters